In [8]:
from functions import *

In [9]:
# Read the CSV file into a DataFrame
result_dir = "../../exp_results"

In [10]:
summary_path = result_dir + "/result_summary.csv"
output_path = result_dir + "/correctness_per_dataset.csv"

check_correntness(summary_path, output_path)

    numTasks  utilization  TP  TN  FP  FN  accuracy
0        3.0          0.2  48   0   0   2      0.96
1        3.0          0.4  44   5   0   1      0.98
2        3.0          0.6  39   9   0   2      0.96
3        6.0          0.2  39   2   0   9      0.82
4        6.0          0.4  35  10   0   5      0.90
5        6.0          0.6  25  19   0   6      0.88
6        9.0          0.2  29   8   0  13      0.74
7        9.0          0.4  25  13   0  12      0.76
8        9.0          0.6  14  32   0   4      0.92
9       12.0          0.2  22   8   0  20      0.60
10      12.0          0.4  18  13   0  19      0.62
11      12.0          0.6   5  37   0   8      0.84


In [11]:
summary_path = result_dir + "/result_summary.csv"

num_cores = 1
num_tasks = 3
utilization =0.4 
detail_result_dir = result_dir + "/detail_result"

FN_subset_info = search_FN_info(summary_path, num_tasks, utilization)
print("FN_info")
print(FN_subset_info)

taskset_indice = FN_subset_info["tasksetIndex"].tolist()
for taskset_index in taskset_indice:
    print("\n\nTaskset_index: ", taskset_index)
    detail_result = show_detail_result(detail_result_dir, num_cores, num_tasks, utilization, taskset_index)
    print(detail_result)

FN_info
    nTasks  util  tasksetIndex  sched(Sim)  sched(proposed)
77       3   0.4            27        True            False


Taskset_index:  27
    WCRT(Sim)  sched(Sim)  WCRT(proposed)  sched(proposed)
id                                                        
1       18397        True           34847            False
2       52473        True           56059             True
3       87569        True           87569             True


In [12]:
# Read the CSV file into a DataFrame
result_dir = "../../exp_results"
summary_path = result_dir + "/result_summary.csv"

# check conservativeness of results for each task
detail_result_dir = result_dir + "/detail_result"
combine_detail_result_path = detail_result_dir + "/combined_detail_result.csv"
combine_detail_result(summary_path,  detail_result_dir, combine_detail_result_path)

output_path = result_dir + "/correctness_per_task.csv"

check_correntness(combine_detail_result_path, output_path)

    numTasks  utilization   TP  TN  FP  FN  accuracy
0        3.0          0.2  148   0   0   2  0.986667
1        3.0          0.4  134   0   0   1  0.992593
2        3.0          0.6  120   0   0   3  0.975610
3        6.0          0.2  279   2   0   7  0.975694
4        6.0          0.4  235   1   0   4  0.983333
5        6.0          0.6  180   2   0   4  0.978495
6        9.0          0.2  364   1   0  13  0.965608
7        9.0          0.4  318   3   0  12  0.963964
8        9.0          0.6  156   0   0   6  0.962963
9       12.0          0.2  476   3   0  25  0.950397
10      12.0          0.4  419   3   0  22  0.950450
11      12.0          0.6  144   2   0  10  0.935897


In [13]:
import pandas as pd

df = pd.read_csv(combine_detail_result_path)
# filter out the tasksets that are not schedulable by the simulator 
TN_df = df[df["simulator_schedulability"] == False]
TN_df

,numCores,numTasks,utilization,tasksetIndex,name,WCRT(sim),simulator_schedulability,WCRT(prop),proposed_schedulability
513,1,6,0.2,19,task4,15058,False,37644,False
644,1,6,0.2,41,task3,10406,False,12653,False
861,1,6,0.4,32,task4,32796,False,63542,False
996,1,6,0.6,18,task1,15373,False,27978,False
1015,1,6,0.6,26,task2,14227,False,22720,False
1348,1,9,0.2,27,task2,31028,False,95247,False
1579,1,9,0.4,10,task8,17236,False,39277,False
1685,1,9,0.4,26,task6,15382,False,42428,False
1727,1,9,0.4,32,task3,13781,False,22563,False
2053,1,12,0.2,4,task11,15086,False,46423,False


In [14]:
summary_df = pd.read_csv(summary_path)
num_cores = 1
num_tasks = 6
utilization = 0.2
taskset_index = 19

# filter out the summary of the taskset with num_cores, num_tasks, utilization, taskset_index
filtered_summary = summary_df[(summary_df["numCores"] == num_cores ) & (summary_df["numTasks"] == num_tasks) & (summary_df["utilization"] == utilization) & (summary_df["tasksetIndex"] == taskset_index)]
filtered_summary

,numCores,numTasks,utilization,tasksetIndex,simulator_schedulability,simulator_timeConsumption(us),proposed_schedulability,proposed_timeConsumption(us)
169,1,6,0.2,19,True,216109,False,409


In [ ]:
filtered_summary = summary_df[(summary_df["simulator_schedulability"] == True) & (summary_df["proposed_schedulability"] == False) ]